## `asnycio` 기초 문법

In [4]:
import time

def count():
    print("One")
    time.sleep(0.5)
    print("Two")
    time.sleep(0.5)

def test():
    for _ in range(3):
        count()

start = time.perf_counter()
test()
elapsed = time.perf_counter() - start
print(f"Elapsed Time: {elapsed:0.2f}초")

One
Two
One
Two
One
Two
Elapsed Time: 3.00초


In [1]:
import asyncio
import time

# Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

async def count():
    print("One")
    await asyncio.sleep(0.5) # 나 0.5초 쉴 건데 다른 일 하고 있어!
    print("Two")
    await asyncio.sleep(0.5) # 나 0.5초 쉴 건데 다른 일 하고 있어!

async def test():
    await asyncio.gather(count(), count(), count())

asyncio.run(test())

One
One
One
Two
Two
Two


## `async`와 `await` 규칙 (이건 꼭 지켜야 해!)

1. **`async def`**: 코루틴(coroutine) 함수를 만들 때 씀.
    * 이 안에서는 `await`, `return`을 쓸 수 있어.


2. **`await`**: "이 작업 끝날 때까지 기다릴게, 그동안 딴 일 해"라는 뜻.
    * **중요:** `await`는 오직 `async def` 함수 안에서만 쓸 수 있어. (일반 함수에서 쓰면 에러 나!)


3. **무엇을 `await` 하나?**: '어웨이터블(Awaitable)' 객체만 기다릴 수 있어. 대부분은 **다른 코루틴**이야.

### 문법 OX 퀴즈

```python
async def f(x):
    y = await z(x)  # (O) 코루틴 안에서 await 사용 가능
    return y

def n(x):
    y = await z(x)  # (X) 일반 함수(def)에서는 await 못 씀! (SyntaxError)
    return y

## 실전 예제: 랜덤 숫자 만들기 (색깔 출력)
비동기의 장점을 보여주는 좀 더 복잡한 예제야. 랜덤한 시간만큼 기다렸다가(I/O 작업 흉내), 랜덤한 숫자를 만드는 코루틴이야.

In [6]:
import asyncio
import random
import colorsys 

# Jupyter Notebook
import nest_asyncio
nest_asyncio.apply()

RESET = "\033[0m"

def get_vibrant_color(idx):
    """
    어두운 배경에서도 잘 보이는 밝고 선명한 색상을 생성합니다.
    True Color(24-bit)를 사용합니다.
    """
    # 황금비(Golden Ratio)를 사용하여 색상을 고르게 분산시킵니다.
    h = (idx * 0.618033988749895) % 1.0
    # h=색상, l=밝기(0.6~0.7이 적당), s=채도(1.0은 아주 선명)
    r, g, b = [int(x * 255) for x in colorsys.hls_to_rgb(h, 0.65, 1.0)]
    
    return f"\033[38;2;{r};{g};{b}m"

async def makerandom(idx, threshold=6):
    # 인덱스에 따라 고유한 색상 생성
    color = get_vibrant_color(idx)
    print(f"{color}작업 {idx} 시작.")
    
    # 0~10 사이 숫자를 뽑는데, threshold보다 작으면 계속 다시 뽑음
    while (number := random.randint(0, 10)) <= threshold:
        print(f"{color}작업 {idx}: {number} (너무 작음, 재시도...)")
        # 잠시 기다리며 제어권 양보 (이때 다른 작업이 실행됨!)
        await asyncio.sleep(idx + 1)
        
    print(f"{color}---> 작업 {idx} 완료! 숫자: {number}")
    return number

async def colorExampleCode():
    # 3개의 makerandom 작업을 동시에 실행
    res = await asyncio.gather(
        makerandom(0, 9),
        makerandom(1, 8),
        makerandom(2, 8),
    )
    return res

random.seed(444)
r1, r2, r3 = asyncio.run(colorExampleCode())
print(f"\n결과: {r1}, {r2}, {r3}")

작업 0 시작.
작업 0: 4 (너무 작음, 재시도...)
작업 1 시작.
작업 1: 4 (너무 작음, 재시도...)
작업 2 시작.
작업 2: 0 (너무 작음, 재시도...)
작업 0: 4 (너무 작음, 재시도...)
작업 1: 7 (너무 작음, 재시도...)
작업 0: 4 (너무 작음, 재시도...)
작업 2: 4 (너무 작음, 재시도...)
작업 0: 8 (너무 작음, 재시도...)
---> 작업 1 완료! 숫자: 10
작업 0: 7 (너무 작음, 재시도...)
작업 0: 8 (너무 작음, 재시도...)
작업 2: 4 (너무 작음, 재시도...)
작업 0: 7 (너무 작음, 재시도...)
작업 0: 1 (너무 작음, 재시도...)
작업 0: 6 (너무 작음, 재시도...)
---> 작업 2 완료! 숫자: 9
작업 0: 3 (너무 작음, 재시도...)
작업 0: 9 (너무 작음, 재시도...)
작업 0: 7 (너무 작음, 재시도...)
---> 작업 0 완료! 숫자: 10

결과: 10, 10, 9


## Event Loop

`asyncio`가 돌아가는 원리에는 이벤트 루프라는 게 있어.

* **역할**: 무한 루프를 돌면서 "지금 쉬고 있는 애 누구니?", "지금 일할 준비 된 애 누구니?"를 감시하고 일을 시키는 관리자야.

* **사용법**: 보통 `asyncio.run(main())`을 하면 파이썬이 알아서 이벤트 루프를 만들고, 실행하고, 끝나면 닫아줘.

* **주의**: 보통 프로그램 하나당 하나의 이벤트 루프(메인 스레드)만 있으면 돼.

`asyncio` **REPL (대화형 쉘)**
파이썬 3.8부터는 터미널에서 `python -m asyncio`를 치면 비동기 전용 쉘이 열려. 여기서는 `asyncio.run()` 없이 바로 `await`를 쳐서 테스트해 볼 수 있어.

In [7]:
await colorExampleCode()

작업 0 시작.
작업 0: 0 (너무 작음, 재시도...)
작업 1 시작.
작업 1: 1 (너무 작음, 재시도...)
작업 2 시작.
작업 2: 0 (너무 작음, 재시도...)
작업 0: 1 (너무 작음, 재시도...)
---> 작업 1 완료! 숫자: 9
작업 0: 0 (너무 작음, 재시도...)
작업 2: 5 (너무 작음, 재시도...)
---> 작업 0 완료! 숫자: 10
작업 2: 5 (너무 작음, 재시도...)
작업 2: 8 (너무 작음, 재시도...)
작업 2: 2 (너무 작음, 재시도...)
작업 2: 5 (너무 작음, 재시도...)
작업 2: 5 (너무 작음, 재시도...)
작업 2: 0 (너무 작음, 재시도...)
작업 2: 3 (너무 작음, 재시도...)
작업 2: 1 (너무 작음, 재시도...)
작업 2: 5 (너무 작음, 재시도...)
작업 2: 6 (너무 작음, 재시도...)
---> 작업 2 완료! 숫자: 10


[10, 9, 10]

## 비동기 프로그래밍 패턴 (실무에서 쓰는 법)

**패턴 1: 코루틴 체이닝 (Chaining) - "릴레이 경주"**   

작업 A의 결과가 작업 B에 필요할 때 써. 사슬처럼 연결된 거지.
* 예: fetch_user(유저 정보 가져오기) -> 그 정보를 이용해 -> fetch_posts(그 유저의 글 가져오기)

In [8]:
import asyncio, random, string, colorsys 

RESET = "\033[0m"

def get_vibrant_color(idx):
    """
    어두운 배경에서도 잘 보이는 밝고 선명한 색상을 생성합니다.
    True Color(24-bit)를 사용합니다.
    """
    # 황금비(Golden Ratio)를 사용하여 색상을 고르게 분산시킵니다.
    h = (idx * 0.618033988749895) % 1.0
    # h=색상, l=밝기(0.6~0.7이 적당), s=채도(1.0은 아주 선명)
    r, g, b = [int(x * 255) for x in colorsys.hls_to_rgb(h, 0.65, 1.0)]
    
    return f"\033[38;2;{r};{g};{b}m"

async def randomStringGenerator(seedString="default", color_idx=2):
    # 1. 필요한 설정 (색상, 지연 시간, 길이)
    color = get_vibrant_color(color_idx)
    delay = 0.5
    length = 10

    # 2. 비동기 대기
    await asyncio.sleep(delay)

    # 3. 무작위 문자열 생성 (알파벳 + 숫자 조합)
    # idx를 생성할 문자열의 길이로 사용합니다.
    letters = string.ascii_letters + string.digits
    random_str = ''.join(random.choice(letters) for _ in range(length))

    # 4. 결과 조합 (seedString과 무작위 문자열 결합)
    result = f"{seedString}_{random_str}"
    
    print(f"[{color}] {delay}초 대기 후 생성 완료: {result}")
    return result


async def fetch_user(user_id):
    return await randomStringGenerator(user_id, 2)

async def fetch_posts(user_info):
    return await randomStringGenerator(user_info, 1)

async def get_user_with_posts(user_id):
    user = await fetch_user(user_id)
    return await fetch_posts(user)

async def userToPost():
    user_ids = ["yabsed1", "yabsed2", "yabsed3"]
    
    # 여기서 코루틴 리스트를 생성합니다.
    tasks = [get_user_with_posts(uid) for uid in user_ids]
    
    # gather를 'await' 하여 결과를 기다립니다.
    return await asyncio.gather(*tasks)


res = asyncio.run(userToPost())
print(f"\n최종 결과: {res}")

[] 0.5초 대기 후 생성 완료: yabsed1_vy5tOIQRC4
[] 0.5초 대기 후 생성 완료: yabsed2_AUJmzNpUk5
[] 0.5초 대기 후 생성 완료: yabsed3_OzOtwdqgHC
[] 0.5초 대기 후 생성 완료: yabsed1_vy5tOIQRC4_qjop2QpLUZ
[] 0.5초 대기 후 생성 완료: yabsed2_AUJmzNpUk5_eoze6ZsSsG
[] 0.5초 대기 후 생성 완료: yabsed3_OzOtwdqgHC_0TbOfofKWz

최종 결과: ['yabsed1_vy5tOIQRC4_qjop2QpLUZ', 'yabsed2_AUJmzNpUk5_eoze6ZsSsG', 'yabsed3_OzOtwdqgHC_0TbOfofKWz']


**패턴 2: 큐 (Queue) 사용 - "생산자와 소비자"**  

이건 서로 모르는 사이일 때 좋아.
* 생산자(Producer): 일을 만들어서 큐(대기열)에 넣음.
* 소비자(Consumer): 큐에서 일을 꺼내서 처리함.

In [9]:
import nest_asyncio
nest_asyncio.apply()

import asyncio

async def producer(queue):
    for i in range(3):
        await asyncio.sleep(0.2)
        await queue.put(f"Item Number {i}")
    await queue.put(None)

async def consumer(queue):
    while True:
        item = await queue.get()
        if item is None:
            break
        print(f"소비자가 {item} 처리 중...")

async def main():
    queue = asyncio.Queue()
    await asyncio.gather(producer(queue), consumer(queue))

asyncio.run(main())

소비자가 Item Number 0 처리 중...
소비자가 Item Number 1 처리 중...
소비자가 Item Number 2 처리 중...


💡 비동기 프로그래밍을 할 때는 **진입점(Entry Point)** 이 되는 `async def main():` **함수를 하나 만들고**, `asyncio.run(main())`으로 실행하는 패턴을 사용하는 것이 가장 안전하고 확실해!

## 기타 강력한 기능들

**1) 비동기 반복문과 컴프리헨션** (`async for`)   

데이터가 한 번에 다 있는 게 아니라, 시간 차를 두고 천천히 들어올 때 써.

In [10]:
async def dataGenerator():
    for i in range(3):
        await asyncio.sleep(0.2)
        yield i

async def dataReceiver():

    async for item in dataGenerator():
        print(item)
        
    data = [i async for i in dataGenerator()]
    print(data)

asyncio.run(dataReceiver())

0
1
2
[0, 1, 2]


**2) 비동기 with 문** (async with)

파일을 열거나 네트워크 연결을 할 때 쓴 후 꼭 닫아줘야 하잖아? 이때 비동기로 처리하려면 `async with`를 써. 대표적으로 `aiohttp`라는 라이브러리로 웹사이트 접속할 때 써.

In [11]:
import aiohttp
import asyncio

async def asyncWith():
    async with aiohttp.ClientSession() as session:
        async with session.get('http://python.org') as response:
            print(response.status)

asyncio.run(asyncWith())

200


**3) 태스크(Task) 관리**

* `asyncio.create_task(coro())`: 코루틴을 즉시 스케줄링(예약)해서 백그라운드에서 실행해버려. 그리고 나중에 await로 결과를 받을 수 있어.

* `asyncio.gather()`: 여러 작업을 한 번에 기다리고, 결과를 리스트로 받음.

* `asyncio.as_completed()`: 여러 작업 중 먼저 끝나는 놈부터 바로바로 처리하고 싶을 때 써.

**4) 예외 처리 (Exception Handling) - Python 3.11+**

여러 코루틴을 동시에 돌리다가 여러 곳에서 에러가 나면? `ExceptionGroup`으로 묶여서 나와. 이걸 잡으려면 `except*` (별표 붙음!)를 써야 해.

```python
try:
    await asyncio.gather(task1(), task2()) # 둘 다 에러 났다고 가정
except* ValueError as e:
    print("ValueError 처리")
except* TypeError as e:
    print("TypeError 처리")

```

## 언제 `asyncio`를 써야 할까?

모든 상황에 좋은 건 아냐. 상황을 잘 봐야 해.

### 👍 `asyncio`가 최고인 경우 (I/O Bound)

* **네트워크 작업:** 웹 서버 만들기, 웹 크롤링, 채팅 프로그램.
* **대기 시간이 많은 작업:** DB 조회, 파일 읽기/쓰기.
* **특징:** CPU는 놀고 있는데 응답을 기다리는 시간이 긴 경우. 스레드를 많이 만드는 것보다 메모리를 훨씬 적게 써서 수천, 수만 개의 연결도 거뜬해.

### 👎 `asyncio`를 쓰면 안 되는 경우 (CPU Bound)

* **복잡한 계산:** 동영상 인코딩, 머신러닝 학습, 수학 계산.
* **이유:** 비동기는 일꾼이 한 명이라고 했지? 혼자서 끙끙대며 계산하느라 다른 일을 전혀 못 해. 이때는 **멀티프로세싱(Multiprocessing)**을 써야 해.
* **주의:** `time.sleep()`이나 일반적인 파일 읽기처럼 코드를 멈추게 하는(Blocking) 함수를 `async` 함수 안에서 쓰면, 프로그램 전체가 멈춰버려!

## `asyncio`를 지원하는 친구들 (라이브러리)

`async` / `await`만 쓴다고 다 되는 게 아냐. 사용하는 도구도 비동기를 지원해야 해.

* **웹 프레임워크:** FastAPI (요즘 대세), Sanic, Quart.
* **네트워크:** aiohttp (웹 요청), httpx.
* **데이터베이스:** Motor (MongoDB), Tortoise ORM.
* **기타:** aiofiles (파일), pytest-asyncio (테스트).

## 요약

1. **Async I/O**는 한 명의 똑똑한 일꾼이 기다리는 시간을 틈타 다른 일을 처리하는 방식이다.
2. **`async def`**로 비동기 함수(코루틴)를 만들고, **`await`**로 실행 및 대기한다.
3. **I/O 바운드 작업(네트워크, 웹 서버 등)**에서 엄청난 성능 향상을 가져온다.
4. 반대로 CPU를 많이 쓰는 계산 작업에는 적합하지 않다.
5. `asyncio.run()`, `gather()`, `create_task()` 등의 도구로 코루틴을 조종한다.

이제 파이썬으로 비동기 프로그래밍을 할 준비가 됐어! 이 기술을 익히면 나중에 수만 명의 사용자를 처리하는 고성능 서버도 만들 수 있단다. 파이팅!